# Notebook 1 — Basic Cleaning for Ads & Feeds (No Merge, No Aggregation)

**Goal**  
Clean the raw datasets so that they are safe and consistent for downstream steps:
- Handle missing values (numeric, categorical, multi-value strings).
- Normalize dtypes (numeric vs categorical).
- Parse time fields to human-usable features.
- Create simple statistics for caret-separated multi-value columns.
- Save clean copies for later notebooks.

**Scope**  
- Clean `train_data_ads.csv` and `test_data_ads.csv`
- Clean `train_data_feeds.csv` and `test_data_feeds.csv`

In [2]:
# 1) Imports & configuration
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)
RANDOM_SEED = 42

## 2) Load all four CSVs

We load training and test files for both domains (ads and feeds).
No transformations yet — just get the data into memory.

In [6]:
# 2) Load raw CSVs (adjust paths if needed)
train_ads  = pd.read_csv('train_data_ads.csv')
test_ads   = pd.read_csv('test_data_ads.csv')
train_feeds= pd.read_csv('train_data_feeds.csv')
test_feeds = pd.read_csv('test_data_feeds.csv')

print("Loaded shapes:",
      "\n  train_ads :", train_ads.shape,
      "\n  test_ads  :", test_ads.shape,
      "\n  train_feeds:", train_feeds.shape,
      "\n  test_feeds :", test_feeds.shape)

Loaded shapes: 
  train_ads : (7675517, 35) 
  test_ads  : (976058, 34) 
  train_feeds: (3227732, 28) 
  test_feeds : (369341, 28)


## 3) Quick audit helper (shape, columns, sample)

A tiny utility to quickly inspect the structure of a DataFrame.
Use it to sanity-check the raw inputs before cleaning.

In [8]:
def quick_peek(name, df, ncols=30, nrows=3):
    print(f'\n===== {name} =====')
    print('Shape:', df.shape)
    print('Columns:', list(df.columns)[:ncols])
    print('\nSample rows:')
    display(df.head(nrows))
    
quick_peek('train_ads (raw)', train_ads)
quick_peek('test_ads (raw)',  test_ads)
quick_peek('train_feeds (raw)', train_feeds)
quick_peek('test_feeds (raw)',  test_feeds)


===== train_ads (raw) =====
Shape: (7675517, 35)
Columns: ['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city', 'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id', 'hispace_app_tags', 'app_second_class', 'app_score', 'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002']

Sample rows:


,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0,100005,3,2,16,147,2,32,6,35,312,2117,7,14339,19319,6,1846,4,46,1,162,47,14,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1,100005,3,2,16,147,2,32,6,35,312,2117,7,11923,21714,8,1060,4,37,1,344,20,13,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1,100005,3,2,16,147,2,32,6,35,312,2117,7,11923,21714,8,1060,4,37,1,344,20,13,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15



===== test_ads (raw) =====
Shape: (976058, 34)
Columns: ['log_id', 'user_id', 'age', 'gender', 'residence', 'city', 'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id', 'hispace_app_tags', 'app_second_class', 'app_score', 'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003']

Sample rows:


,log_id,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,242295,100002,2,3,13,225,2,31,3,21,210,2177,4,14584,17683,5,1236,3,22,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101201,86^15^86^112^191,0,15
1,242300,100002,2,3,13,225,2,31,3,21,210,2419,6,14584,17683,5,1236,3,13,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101154,86^15^86^112^191,0,15
2,242299,100002,2,3,13,225,2,31,3,21,210,2177,4,35178,10779,8,1060,4,58,1,344,20,13,10.0,21791^33265,2020^1236,352,24107,1218,173,202206101228,86^15^86^112^191,0,15



===== train_feeds (raw) =====
Shape: (3227732, 28)
Columns: ['u_userId', 'u_phonePrice', 'u_browserLifeCycle', 'u_browserMode', 'u_feedLifeCycle', 'u_refreshTimes', 'u_newsCatInterests', 'u_newsCatDislike', 'u_newsCatInterestsST', 'u_click_ca2_news', 'i_docId', 'i_s_sourceId', 'i_regionEntity', 'i_cat', 'i_entities', 'i_dislikeTimes', 'i_upTimes', 'i_dtype', 'e_ch', 'e_m', 'e_po', 'e_pl', 'e_rn', 'e_section', 'e_et', 'label', 'cillabel', 'pro']

Sample rows:


,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,i_docId,i_s_sourceId,i_regionEntity,i_cat,i_entities,i_dislikeTimes,i_upTimes,i_dtype,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,6e3e6bcb58d1fdd57b01a423f7a777a51cbcc00e,1eba881ea576fc05dfe457dd2111cd1505caa5a8,0,210,c7c6575e357e3e82c87fd04b04b28ccb878653d1c68b3a...,4,1,13,19,1217,1,561,2,0,202206081521,-1,-1,0
1,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,f080276ae8aee56306253debbf643fb34b5af697,772b4a2593c3aa2590291f25d4bbc463a9ac49a2,0,219,40179dae61fecd7eaad86cdd128e70f119dd323750fc23...,3,9,13,19,1217,9,561,1,0,202206081521,-1,-1,0
2,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,f6ee51f33fe8623b713a16287a873c98b57e6861,645e57afbbafce5159ab7fe956837c40103f33c9,0,78,37d6e916f8c486d37683f03df3f578a6a65ac9fff01d03...,0,9,13,19,1217,18,561,1,0,202206081521,-1,-1,0



===== test_feeds (raw) =====
Shape: (369341, 28)
Columns: ['u_userId', 'u_phonePrice', 'u_browserLifeCycle', 'u_browserMode', 'u_feedLifeCycle', 'u_refreshTimes', 'u_newsCatInterests', 'u_newsCatDislike', 'u_newsCatInterestsST', 'u_click_ca2_news', 'i_docId', 'i_s_sourceId', 'i_regionEntity', 'i_cat', 'i_entities', 'i_dislikeTimes', 'i_upTimes', 'i_dtype', 'e_ch', 'e_m', 'e_po', 'e_pl', 'e_rn', 'e_section', 'e_et', 'label', 'cillabel', 'pro']

Sample rows:


,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,i_docId,i_s_sourceId,i_regionEntity,i_cat,i_entities,i_dislikeTimes,i_upTimes,i_dtype,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,215116,16,17,14,11,0,65^72^42^65^104,0,219^50^151^21^140,219^50^151^114^21,d676da312ea5fdce3f4ee5ebe9c83735ee48a2a8,121eacca77fc36a02d7cb6be12d8a9f3b369905a,0,123,44b369eeae2d16ecc1f4cf65ce925b0e22f36abde9cd84...,9,9,12,19,998,9,2449,13,0,202206100922,-1,-1,0
1,215116,16,17,14,11,0,65^72^42^65^104,0,219^50^151^21^140,219^50^151^114^21,b8532b78aab908ef4cbdb068f2317fb73bf38f54,c4d3ed25402e08c1d3633c806bc0987219a36edb,0,219,e7543ba582daa6a6e6f06f3b5513eef62ea2ff5b205b2b...,4,9,12,19,998,7,2449,14,0,202206100924,-1,-1,0
2,215116,16,17,14,11,0,65^72^42^65^104,0,0,65^104^98^168^0,be4e43d78d637723b15bba344131bf0c66d964d7,3a705744387ea91d68ce6b8172b528c53df49185,0,106,NaN,9,9,12,19,998,14,2449,1,0,202206100907,1,-1,40


## 4) Define common cleaning rules (R1–R5)

**Missing values (R1).**  
- Numeric → fillna(-1)  
- Categorical → fillna("unknown")  
- Multi-value strings (caret `^` separated) → fillna("")

**Dtype normalization (R2).**  
- IDs/enums: as `category` (later easy to encode)
- True numeric: keep as `int/float`
- Times: keep raw + derive features

**Time features (R3).**  
- From `pt_d` (ads): hour, weekday, weekend flag  
- From `e_et` (feeds): hour, weekday, weekend flag  
- (Optional) hour_sin, hour_cos if needed later

**High cardinality (R4).**  
- We only mark columns now; truncation/label encoding will be handled in Notebook 3 after merge.

**Multi-value strings (R5).**  
- Create simple statistics only: `<col>_len` and `<col>_uniq`

## 5) Cleaning helpers

We define small helper functions that:
- detect presence of a column
- fill missing values according to type
- parse time columns
- compute stats for caret-separated strings

In [10]:
def has(df, col):
    return col in df.columns

def fill_missing_basic(df, label_col=None, mv_cols=None):
    """R1: fill NaNs by dtype; mv_cols get empty-string fills."""
    mv_cols = mv_cols or []
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # multi-value string columns
    for c in [c for c in mv_cols if has(df, c)]:
        df[c] = df[c].fillna("")
    # other object columns
    for c in [c for c in obj_cols if c not in mv_cols]:
        df[c] = df[c].fillna("unknown")
    # numeric columns
    for c in num_cols:
        if label_col and c == label_col:
            continue
        df[c] = df[c].fillna(-1)
    return df

def parse_ads_time(df, time_col='pt_d'):
    """R3: parse ads time like 202206030326 → hour/wday/weekend."""
    if not has(df, time_col):
        return df
    s = df[time_col].astype(str).str.zfill(12)
    dt = pd.to_datetime(s, format='%Y%m%d%H%M', errors='coerce')
    df['t_hour'] = dt.dt.hour.fillna(-1).astype('int16')
    df['t_wday'] = dt.dt.weekday.fillna(-1).astype('int8')  # Mon=0
    df['t_is_weekend'] = df['t_wday'].isin([5,6]).astype('int8')
    return df

def parse_feeds_time(df, time_col='e_et'):
    """R3: parse feeds time like 202206081521 → hour/wday/weekend."""
    if not has(df, time_col):
        return df
    s = df[time_col].astype(str).str.zfill(12)
    dt = pd.to_datetime(s, format='%Y%m%d%H%M', errors='coerce')
    df['t_hour'] = dt.dt.hour.fillna(-1).astype('int16')
    df['t_wday'] = dt.dt.weekday.fillna(-1).astype('int8')
    df['t_is_weekend'] = df['t_wday'].isin([5,6]).astype('int8')
    return df

def mv_len(s: str) -> int:
    if not s or s == "0":
        return 0
    return len(s.split('^'))

def mv_uniq(s: str) -> int:
    if not s or s == "0":
        return 0
    return len(set(s.split('^')))

def add_mv_stats(df, cols):
    """R5: add <col>_len and <col>_uniq for caret-separated lists."""
    for c in cols:
        if has(df, c):
            df[c+'_len']  = df[c].astype(str).apply(mv_len).astype('int32')
            df[c+'_uniq'] = df[c].astype(str).apply(mv_uniq).astype('int32')
    return df

## 6) Clean `train_data_ads` (no aggregation, no merge)

**Goal**  
Prepare a clean working copy of the ads training table so it’s safe for downstream steps:

1. Detect caret-separated multi-value columns present in this dataset.
2. Apply missing-value rules (R1).
3. Parse the time column `pt_d` → `t_hour`, `t_wday`, `t_is_weekend` (R3).
4. Add simple stats for multi-value columns: `<col>_len`, `<col>_uniq` (R5).
5. Optionally cast a few low-cardinality columns to `category` (R2).
6. Run quick sanity checks (label distribution, preview of new columns).

In [12]:
# 6) Clean train_data_ads
LABEL_COL = 'label'
ads_tr = train_ads.copy()  # keep original train_ads intact

# 6.1 detect multi-value columns present in ads (caret-separated strings)
mv_candidates_ads = [
    'ad_click_list_v001','ad_click_list_v002','ad_click_list_v003',
    'ad_close_list_v001','ad_close_list_v002','ad_close_list_v003',
    'u_newsCatInterestsST'  # sometimes short-term interests string
]
mv_cols_ads = [c for c in mv_candidates_ads if has(ads_tr, c)]
print("Multi-value columns in train_ads:", mv_cols_ads)

Multi-value columns in train_ads: ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003', 'u_newsCatInterestsST']


In [14]:
# 6.2 apply missing-value rules (R1)
ads_tr = fill_missing_basic(ads_tr, label_col=LABEL_COL, mv_cols=mv_cols_ads)

# 6.3 parse time column pt_d → hour / weekday / weekend (R3)
ads_tr = parse_ads_time(ads_tr, time_col='pt_d')

# 6.4 add simple stats for multi-value columns (R5)
ads_tr = add_mv_stats(ads_tr, mv_cols_ads)

# 6.5 cast a few low-cardinality columns to 'category' (R2)
low_card_candidates = ['gender','age','city_rank','net_type','creat_type_cd','inter_type_cd']
for c in [c for c in low_card_candidates if has(ads_tr, c)]:
    ads_tr[c] = ads_tr[c].astype('category')

# 6.6 sanity checks: label distribution + preview new columns
print("\n[train_ads] Label distribution:")
if has(ads_tr, LABEL_COL):
    print(ads_tr[LABEL_COL].value_counts(normalize=True))
else:
    print("WARNING: label column not found!")

new_cols_ads = []
if has(ads_tr, 't_hour'):
    new_cols_ads += ['t_hour','t_wday','t_is_weekend']
new_cols_ads += [c+'_len' for c in mv_cols_ads] + [c+'_uniq' for c in mv_cols_ads]

print("\n[train_ads] Newly created columns:", new_cols_ads[:20], ("... total: "+str(len(new_cols_ads)) if len(new_cols_ads)>20 else ""))

# Show first few rows with only the newly derived columns + label for a quick look
display_cols = ([LABEL_COL] if has(ads_tr, LABEL_COL) else []) + new_cols_ads
display_cols = [c for c in display_cols if c in ads_tr.columns]
ads_tr[display_cols].head(5)


[train_ads] Label distribution:
label
0    0.984478
1    0.015522
Name: proportion, dtype: float64

[train_ads] Newly created columns: ['t_hour', 't_wday', 't_is_weekend', 'ad_click_list_v001_len', 'ad_click_list_v002_len', 'ad_click_list_v003_len', 'ad_close_list_v001_len', 'ad_close_list_v002_len', 'ad_close_list_v003_len', 'u_newsCatInterestsST_len', 'ad_click_list_v001_uniq', 'ad_click_list_v002_uniq', 'ad_click_list_v003_uniq', 'ad_close_list_v001_uniq', 'ad_close_list_v002_uniq', 'ad_close_list_v003_uniq', 'u_newsCatInterestsST_uniq'] 


,label,t_hour,t_wday,t_is_weekend,ad_click_list_v001_len,ad_click_list_v002_len,ad_click_list_v003_len,ad_close_list_v001_len,ad_close_list_v002_len,ad_close_list_v003_len,u_newsCatInterestsST_len,ad_click_list_v001_uniq,ad_click_list_v002_uniq,ad_click_list_v003_uniq,ad_close_list_v001_uniq,ad_close_list_v002_uniq,ad_close_list_v003_uniq,u_newsCatInterestsST_uniq
0,0,3,4,0,4,3,3,1,1,1,3,4,3,3,1,1,1,3
1,1,3,4,0,4,3,3,1,1,1,3,4,3,3,1,1,1,3
2,1,3,4,0,4,3,3,1,1,1,3,4,3,3,1,1,1,3
3,0,3,4,0,4,3,3,1,1,1,3,4,3,3,1,1,1,3
4,0,3,4,0,4,3,3,1,1,1,3,4,3,3,1,1,1,3


## 7) Clean `test_data_ads` (same rules, no label)

**Goal**  
Apply exactly the same cleaning logic to the ads *test* dataset so that  
its structure, datatypes, and feature names perfectly match the training ads table.

Steps:  
1. Copy the raw test_ads.  
2. Apply missing-value filling (R1).  
3. Parse time `pt_d` (R3).  
4. Add multi-value stats (R5).  
5. Cast low-cardinality columns to `category` (R2).  
6. Sanity-check column names and dtypes to confirm parity with train_ads.

In [16]:
# 7) Clean test_data_ads
ads_te = test_ads.copy()

# multi-value columns: same candidates as training
mv_candidates_ads = [
    'ad_click_list_v001','ad_click_list_v002','ad_click_list_v003',
    'ad_close_list_v001','ad_close_list_v002','ad_close_list_v003',
    'u_newsCatInterestsST'
]
mv_cols_ads_te = [c for c in mv_candidates_ads if has(ads_te, c)]

# fill missing values
ads_te = fill_missing_basic(ads_te, label_col=None, mv_cols=mv_cols_ads_te)

# parse time column
ads_te = parse_ads_time(ads_te, time_col='pt_d')

# add simple stats for multi-value columns
ads_te = add_mv_stats(ads_te, mv_cols_ads_te)

# cast same low-cardinality columns
low_card_candidates = ['gender','age','city_rank','net_type','creat_type_cd','inter_type_cd']
for c in [c for c in low_card_candidates if has(ads_te, c)]:
    ads_te[c] = ads_te[c].astype('category')

# final structure check
print("test_data_ads cleaned successfully.")
print("Shape:", ads_te.shape)
print("Newly created columns:", [c for c in ads_te.columns if any(x in c for x in ['t_hour','_len','_uniq'])][:20])
quick_peek('test_ads (clean basic)', ads_te)

test_data_ads cleaned successfully.
Shape: (976058, 51)
Newly created columns: ['t_hour', 'ad_click_list_v001_len', 'ad_click_list_v001_uniq', 'ad_click_list_v002_len', 'ad_click_list_v002_uniq', 'ad_click_list_v003_len', 'ad_click_list_v003_uniq', 'ad_close_list_v001_len', 'ad_close_list_v001_uniq', 'ad_close_list_v002_len', 'ad_close_list_v002_uniq', 'ad_close_list_v003_len', 'ad_close_list_v003_uniq', 'u_newsCatInterestsST_len', 'u_newsCatInterestsST_uniq']

===== test_ads (clean basic) =====
Shape: (976058, 51)
Columns: ['log_id', 'user_id', 'age', 'gender', 'residence', 'city', 'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id', 'hispace_app_tags', 'app_second_class', 'app_score', 'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003']

Sample rows:


,log_id,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle,t_hour,t_wday,t_is_weekend,ad_click_list_v001_len,ad_click_list_v001_uniq,ad_click_list_v002_len,ad_click_list_v002_uniq,ad_click_list_v003_len,ad_click_list_v003_uniq,ad_close_list_v001_len,ad_close_list_v001_uniq,ad_close_list_v002_len,ad_close_list_v002_uniq,ad_close_list_v003_len,ad_close_list_v003_uniq,u_newsCatInterestsST_len,u_newsCatInterestsST_uniq
0,242295,100002,2,3,13,225,2,31,3,21,210,2177,4,14584,17683,5,1236,3,22,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101201,86^15^86^112^191,0,15,12,4,0,2,2,2,2,1,1,1,1,1,1,1,1,5,4
1,242300,100002,2,3,13,225,2,31,3,21,210,2419,6,14584,17683,5,1236,3,13,1,213,18,23,0.0,21791^33265,2020^1236,352,24107,1218,173,202206101154,86^15^86^112^191,0,15,11,4,0,2,2,2,2,1,1,1,1,1,1,1,1,5,4
2,242299,100002,2,3,13,225,2,31,3,21,210,2177,4,35178,10779,8,1060,4,58,1,344,20,13,10.0,21791^33265,2020^1236,352,24107,1218,173,202206101228,86^15^86^112^191,0,15,12,4,0,2,2,2,2,1,1,1,1,1,1,1,1,5,4


## 8) Clean `train_data_feeds` (no aggregation yet)

**Goal**  
Perform the same basic cleaning on the *feeds* training dataset —  
so that it becomes consistent, clean, and ready for aggregation in the next notebook.

Steps:
1. Copy the raw feeds training data.
2. Identify caret-separated multi-value columns (like user interests or entities).
3. Apply missing-value rules (R1).
4. Parse the time column `e_et` → `t_hour`, `t_wday`, `t_is_weekend` (R3).
5. Add `<col>_len` and `<col>_uniq` features for multi-value string columns (R5).
6. Convert some low-cardinality categorical columns to `category` (R2).
7. Check shape, new columns, and sample output.

In [18]:
# 8) Clean train_data_feeds
feeds_tr = train_feeds.copy()

# 8.1 identify multi-value columns (caret-separated)
mv_candidates_feeds = [
    'u_newsCatInterests',     
    'u_newsCatInterestsST',   
    'u_click_ca2_news',       
    'i_entities'             
]
mv_cols_feeds = [c for c in mv_candidates_feeds if has(feeds_tr, c)]
print("Multi-value columns in train_feeds:", mv_cols_feeds)

Multi-value columns in train_feeds: ['u_newsCatInterests', 'u_newsCatInterestsST', 'u_click_ca2_news', 'i_entities']


In [20]:
# 8.2 apply missing-value rules (R1)
feeds_tr = fill_missing_basic(feeds_tr, label_col=None, mv_cols=mv_cols_feeds)

# 8.3 parse time column 'e_et' → t_hour / t_wday / t_is_weekend
feeds_tr = parse_feeds_time(feeds_tr, time_col='e_et')

# 8.4 add stats for multi-value columns (R5)
feeds_tr = add_mv_stats(feeds_tr, mv_cols_feeds)

# 8.5 cast a few low-cardinality categorical columns to category (R2)
low_card_feeds = ['u_browserMode','u_browserLifeCycle','i_dtype','e_section','e_ch']
for c in [c for c in low_card_feeds if has(feeds_tr, c)]:
    feeds_tr[c] = feeds_tr[c].astype('category')

# 8.6 sanity check: shape, new columns, sample
print("train_data_feeds cleaned successfully.")
print("Shape:", feeds_tr.shape)

new_cols_feeds = []
if has(feeds_tr, 't_hour'):
    new_cols_feeds += ['t_hour','t_wday','t_is_weekend']
new_cols_feeds += [c+'_len' for c in mv_cols_feeds] + [c+'_uniq' for c in mv_cols_feeds]
print("Newly created columns:", new_cols_feeds)

# Show sample of only new derived columns
display_cols = [c for c in new_cols_feeds if c in feeds_tr.columns]
quick_peek('train_feeds (clean basic)', feeds_tr[display_cols + (['u_userId'] if has(feeds_tr, 'u_userId') else [])])

train_data_feeds cleaned successfully.
Shape: (3227732, 39)
Newly created columns: ['t_hour', 't_wday', 't_is_weekend', 'u_newsCatInterests_len', 'u_newsCatInterestsST_len', 'u_click_ca2_news_len', 'i_entities_len', 'u_newsCatInterests_uniq', 'u_newsCatInterestsST_uniq', 'u_click_ca2_news_uniq', 'i_entities_uniq']

===== train_feeds (clean basic) =====
Shape: (3227732, 12)
Columns: ['t_hour', 't_wday', 't_is_weekend', 'u_newsCatInterests_len', 'u_newsCatInterestsST_len', 'u_click_ca2_news_len', 'i_entities_len', 'u_newsCatInterests_uniq', 'u_newsCatInterestsST_uniq', 'u_click_ca2_news_uniq', 'i_entities_uniq', 'u_userId']

Sample rows:


,t_hour,t_wday,t_is_weekend,u_newsCatInterests_len,u_newsCatInterestsST_len,u_click_ca2_news_len,i_entities_len,u_newsCatInterests_uniq,u_newsCatInterestsST_uniq,u_click_ca2_news_uniq,i_entities_uniq,u_userId
0,15,2,0,5,5,5,5,5,5,5,5,135880
1,15,2,0,5,5,5,5,5,5,5,5,135880
2,15,2,0,5,5,5,5,5,5,5,5,135880


## 9) Clean `test_data_feeds` (same rules as training feeds)

**Goal**  
Apply the same cleaning logic used for `train_data_feeds`,  
so that test feeds data has the exact same structure and feature columns.

Steps:
1. Copy raw test feeds data.
2. Identify caret-separated multi-value columns.
3. Apply missing-value rules (R1).
4. Parse time `e_et` → hour / weekday / weekend (R3).
5. Add `<col>_len` and `<col>_uniq` stats (R5).
6. Convert low-cardinality columns to `category` (R2).
7. Confirm column names and dtypes match training feeds.

In [22]:
# 9) Clean test_data_feeds
feeds_te = test_feeds.copy()

# 9.1 identify multi-value columns
mv_candidates_feeds = [
    'u_newsCatInterests',
    'u_newsCatInterestsST',
    'u_click_ca2_news',
    'i_entities'
]
mv_cols_feeds_te = [c for c in mv_candidates_feeds if has(feeds_te, c)]
print("Multi-value columns in test_feeds:", mv_cols_feeds_te)

Multi-value columns in test_feeds: ['u_newsCatInterests', 'u_newsCatInterestsST', 'u_click_ca2_news', 'i_entities']


In [24]:
# 9.2 apply missing-value rules
feeds_te = fill_missing_basic(feeds_te, label_col=None, mv_cols=mv_cols_feeds_te)

# 9.3 parse time column 'e_et'
feeds_te = parse_feeds_time(feeds_te, time_col='e_et')

# 9.4 add stats for multi-value columns
feeds_te = add_mv_stats(feeds_te, mv_cols_feeds_te)

# 9.5 cast a few low-cardinality categorical columns
low_card_feeds = ['u_browserMode','u_browserLifeCycle','i_dtype','e_section','e_ch']
for c in [c for c in low_card_feeds if has(feeds_te, c)]:
    feeds_te[c] = feeds_te[c].astype('category')

# 9.6 sanity check
print("test_data_feeds cleaned successfully.")
print("Shape:", feeds_te.shape)

new_cols_feeds = []
if has(feeds_te, 't_hour'):
    new_cols_feeds += ['t_hour','t_wday','t_is_weekend']
new_cols_feeds += [c+'_len' for c in mv_cols_feeds_te] + [c+'_uniq' for c in mv_cols_feeds_te]
print("Newly created columns:", new_cols_feeds)

# show a few derived columns to verify
display_cols = [c for c in new_cols_feeds if c in feeds_te.columns]
quick_peek('test_feeds (clean basic)', feeds_te[display_cols + (['u_userId'] if has(feeds_te, 'u_userId') else [])])

test_data_feeds cleaned successfully.
Shape: (369341, 39)
Newly created columns: ['t_hour', 't_wday', 't_is_weekend', 'u_newsCatInterests_len', 'u_newsCatInterestsST_len', 'u_click_ca2_news_len', 'i_entities_len', 'u_newsCatInterests_uniq', 'u_newsCatInterestsST_uniq', 'u_click_ca2_news_uniq', 'i_entities_uniq']

===== test_feeds (clean basic) =====
Shape: (369341, 12)
Columns: ['t_hour', 't_wday', 't_is_weekend', 'u_newsCatInterests_len', 'u_newsCatInterestsST_len', 'u_click_ca2_news_len', 'i_entities_len', 'u_newsCatInterests_uniq', 'u_newsCatInterestsST_uniq', 'u_click_ca2_news_uniq', 'i_entities_uniq', 'u_userId']

Sample rows:


,t_hour,t_wday,t_is_weekend,u_newsCatInterests_len,u_newsCatInterestsST_len,u_click_ca2_news_len,i_entities_len,u_newsCatInterests_uniq,u_newsCatInterestsST_uniq,u_click_ca2_news_uniq,i_entities_uniq,u_userId
0,9,4,0,5,5,5,1,4,5,5,1,215116
1,9,4,0,5,5,5,5,4,5,5,5,215116
2,9,4,0,5,0,5,0,4,0,5,0,215116


## 10) Save cleaned datasets for Notebook 2 & 3

**Goal**  
Persist clean-but-unaggregated copies to disk so that the next notebooks can load them reliably.

In [27]:
# Save cleaned copies (from steps you've already run)
ads_tr.to_parquet('ads_train_clean.parquet', index=False)
ads_te.to_parquet('ads_test_clean.parquet', index=False)
feeds_tr.to_parquet('feeds_train_clean.parquet', index=False)
feeds_te.to_parquet('feeds_test_clean.parquet', index=False)

print("Saved files:")
print(" - ads_train_clean.parquet")
print(" - ads_test_clean.parquet")
print(" - feeds_train_clean.parquet")
print(" - feeds_test_clean.parquet")

Saved files:
 - ads_train_clean.parquet
 - ads_test_clean.parquet
 - feeds_train_clean.parquet
 - feeds_test_clean.parquet
